In [1]:
!pip install sklearn_crfsuite;
#!pip install eli5;
#!pip install --force-reinstall --no-dependencies "scikit-learn==0.24.2";

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import nltk
import sklearn_crfsuite
#import eli5
import pandas as pd
import csv

from itertools import chain

import nltk
import sklearn
import scipy.stats
#from sklearn.metrics import make_scorer
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import RandomizedSearchCV

#from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [3]:
RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

if RunningInCOLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP/KIND-main')
    os.getcwd()

    print("Colab environment detected. Paths have been set.")
else:
    print("You are not running in Google Colab. Skipping this step...")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colab environment detected. Paths have been set.


In [4]:
file_path_loc = os.path.join(os.getcwd(), 'gazetteers/it-LOC-wikipedia.txt')
file_path_org = os.path.join(os.getcwd(), 'gazetteers/it-ORG-wikipedia.txt')
file_path_per = os.path.join(os.getcwd(), 'gazetteers/it-PER-wikipedia.txt')

with open(file_path_loc, 'r') as f_loc, open(file_path_org, 'r') as f_org, open(file_path_per, 'r') as f_per:
        gaz_loc = [line[4:-1] for line in f_loc.readlines()]
        gaz_org = [line[4:-1] for line in f_org.readlines()]
        gaz_per = [line[4:-1] for line in f_per.readlines()]

In [5]:
# mettere in set
# mettere postag con spacy
# problema perchè come controllo una parola composta? magari mettere in word+1 la combinazione delle due se è in qualche gaz 
# oppure tokenizzare per parola e togliere stopwords e punteggiatura
print(gaz_loc[:3])
print(gaz_org[:3])
print(gaz_per[:3])

['Irlanda del Nord', 'nordirlandese', 'Irlanda del nord']
['Boeing', 'Boeing Commercial Airplanes', 'Boeing Company']
['George Washington', 'Giorgio Washington', 'Jack Bauer']


Import of train and test dataset. Different combinations of datasets are possible.

In [6]:
# Path of training file
file_path_train = os.path.join(os.getcwd(), 'dataset/wikinews_train.tsv')

# Inserting the content of the tsv file into a dataframe keeping the blank lines
data_df_train = pd.read_csv(file_path_train, sep='\t', names=['Word', 'Entity'], skip_blank_lines=False, quoting=csv.QUOTE_NONE)

# Replace NaN values with a new line (\n) to mark the beginning of a new phrase
data_df_train.fillna('\n', inplace=True)

# Path of file degasperi_test.tsv
file_path_test = os.path.join(os.getcwd(), 'dataset/wikinews_test.tsv')

# Inserting the content of the tsv file into a dataframe keeping the blank lines (i.e. the end of a sentence)
data_df_test = pd.read_csv(file_path_test, sep='\t', names=['Word', 'Entity'], skip_blank_lines=False, quoting=csv.QUOTE_NONE)

# Replace NaN values with a new line (\n) to mark the beginning of a new phrase
data_df_test.fillna('\n', inplace=True)

In [7]:
# See first elements
data_df_train[:12]

,Word,Entity
0,In,O
1,fiamme,O
2,l',O
3,Istituto,LOC
4,Lama,LOC
5,Tzong,LOC
6,Khapa,LOC
7,",",O
8,monastero,O
9,buddhista,O


In [8]:
def sentences_transform(data_df):
  """
  Given in input a dataframe containing words and entities,
  returns a list of tuples where each tuple is associated to a sentence
  and has as first element a list of words and as second element a list of
  associated entities.
  """

  # Group the DataFrame by sentence using the '\n' separator
  grouped = data_df_train.groupby(data_df['Word'].eq('\n').cumsum())

  # Iterate over the groups and concatenate the words and tags into sentences
  sentences = []
  for _, group in grouped:
      sentence = ' '.join(group['Word'])
      tags = ' '.join(group['Entity'])
      sentences.append((sentence.strip().split(), tags.strip().split()))
      
  return sentences

In [9]:
sentences_train = sentences_transform(data_df_train)
sentences_test = sentences_transform(data_df_test)
print(sentences_train[:2])

[(['In', 'fiamme', "l'", 'Istituto', 'Lama', 'Tzong', 'Khapa', ',', 'monastero', 'buddhista'], ['O', 'O', 'O', 'LOC', 'LOC', 'LOC', 'LOC', 'O', 'O', 'O']), (['In', 'fiamme', 'il', 'Monastero', 'Lama', 'Tzong', 'Khapa', ',', 'centro', 'buddhista', 'di', 'Pisa', ',', 'una', 'delle', 'istituzioni', 'più', 'importanti', 'per', 'lo', 'studio', 'e', 'la', 'pratica', 'del', 'Buddhismo', 'di', 'tradizione', 'tibetana', 'in', 'tutta', 'Europa', '.'], ['O', 'O', 'O', 'LOC', 'LOC', 'LOC', 'LOC', 'O', 'O', 'O', 'O', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOC', 'O'])]


In [10]:
sentences_train[0][1]

['O', 'O', 'O', 'LOC', 'LOC', 'LOC', 'LOC', 'O', 'O', 'O']

In [11]:
# Each sentence should be converted to a list of dict
def word2features(sent, i):
    word = sent[i]
    #postag = sent[i][1] Don't have postags at the moment

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(), # Check if start with an upper case
        'word.isdigit()': word.isdigit(),
        'word.inloc()': word in gaz_loc,
        'word.inorg()': word in gaz_org,
        'word.inper()': word in gaz_per,
        #'postag': postag,
        #'postag[:2]': postag[:2],
    }
    if i > 0:
        # Take previous word of the sentence
        word1 = sent[i-1]
        #postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            #'-1:postag': postag1,
            #'-1:postag[:2]': postag1[:2],
        })
    else:
        # BOS = beginning of string? Però così non tutti ce l'hanno
        features['BOS'] = True

    if i < len(sent)-1:
        # Take following word of the sentence
        word1 = sent[i+1]
        #postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            #'+1:postag': postag1,
            #'+1:postag[:2]': postag1[:2],
        })
    else:
      # EOS = end of string?
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent[0], i) for i in range(len(sent[0]))]

def sent2labels(sent):
    return sent[1]

def sent2tokens(sent):
    return sent[0]

In [12]:
gaz_loc = set(gaz_loc)
gaz_org = set(gaz_org)
gaz_per = set(gaz_per)

In [13]:
# Convert dataset in the right format
X_train = [sent2features(sentence) for sentence in sentences_train]
y_train = [sent2labels(sentence) for sentence in sentences_train]

X_test = [sent2features(sentence) for sentence in sentences_test]
y_test = [sent2labels(sentence) for sentence in sentences_test]

In [14]:
# Check features for the first word of first sentence
X_train[0][0]

{'bias': 1.0,
 'word.lower()': 'in',
 'word[-3:]': 'In',
 'word[-2:]': 'In',
 'word.isupper()': False,
 'word.istitle()': True,
 'word.isdigit()': False,
 'word.inloc()': False,
 'word.inorg()': False,
 'word.inper()': False,
 'BOS': True,
 '+1:word.lower()': 'fiamme',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False}

In [15]:
# Define the training algorithm
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True, # slightly better with True
)

# Train the model
crf.fit(X_train, y_train);

In [16]:
crf.classes_

['O', 'LOC', 'PER', 'ORG']

In [17]:
# Remove 'O' label, we are not interested in it
labels = list(crf.classes_)
labels.remove('O')
labels

['LOC', 'PER', 'ORG']

In [18]:
# Compute predictions
y_pred = crf.predict(X_test)

# Compute 
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.9424912263923237

In [19]:
from sklearn.preprocessing import LabelEncoder

# create the vector of string labels
labels = ['O', 'PER', 'LOC', 'ORG']

# create a LabelEncoder object and fit_transform the labels
le = LabelEncoder()
le.fit(labels)
numeric_labels = le.transform(labels)


print(numeric_labels)
le.classes_

[1 3 0 2]


array(['LOC', 'O', 'ORG', 'PER'], dtype='<U3')

In [20]:
y_pred_num = [le.transform(sent) for sent in y_pred]
y_test_num = [le.transform(sent) for sent in y_test]

In [21]:
from pandas.core.common import flatten

y_pred_num = list(flatten(y_pred_num))
y_test_num = list(flatten(y_test_num))

In [22]:
print(sklearn.metrics.classification_report(y_test_num, y_pred_num, labels=[0,2,3], digits=3))

              precision    recall  f1-score   support

           0      0.908     0.947     0.927      1660
           2      0.950     0.848     0.896      1653
           3      0.989     0.982     0.985      2379

   micro avg      0.954     0.933     0.943      5692
   macro avg      0.949     0.925     0.936      5692
weighted avg      0.954     0.933     0.942      5692



In [23]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[::-1])

Top likely transitions:
ORG    -> ORG     3.882359
LOC    -> LOC     3.536723
O      -> O       2.535663
PER    -> PER     1.908443
O      -> ORG     -0.443471
O      -> LOC     -0.677226
O      -> PER     -0.906261
ORG    -> O       -1.082594
ORG    -> PER     -1.271687
PER    -> O       -1.413751
LOC    -> O       -1.649565
PER    -> ORG     -3.026632
LOC    -> PER     -3.162489
LOC    -> ORG     -3.771079
PER    -> LOC     -4.484841
ORG    -> LOC     -4.851020

Top unlikely transitions:
ORG    -> LOC     -4.851020
PER    -> LOC     -4.484841
LOC    -> ORG     -3.771079
LOC    -> PER     -3.162489
PER    -> ORG     -3.026632
LOC    -> O       -1.649565
PER    -> O       -1.413751
ORG    -> PER     -1.271687
ORG    -> O       -1.082594
O      -> PER     -0.906261
O      -> LOC     -0.677226
O      -> ORG     -0.443471
PER    -> PER     1.908443
O      -> O       2.535663
LOC    -> LOC     3.536723
ORG    -> ORG     3.882359
